# Create customer order summary
1. Join the tables silver_customers, silver_addresses, and silver_orders
2. Retrieve the latest address of the customer
3. Calculate the following views
* total_orders
* total_items_ordered
* total_order_amount

In [0]:
import dlt
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
@dlt.materialized_view(
    name='gold_customer_order_summary'
)
def create_gold_customer_order_summary():
    df_customers = spark.read.table('LIVE.silver_customers')
    df_addresses = spark.read.table('LIVE.silver_addresses')
    df_orders = spark.read.table('LIVE.silver_orders')

    # summarize only completed orders
    df = df_customers.join(df_orders, df_customers.customer_id == df_orders.customer_id)\
                    .join(df_addresses, df_customers.customer_id == df_addresses.customer_id)\
                    .filter(
                        (df_addresses.__END_AT.isNull()) & (df_orders.order_status == 'Completed')
                        )\
                    .groupby(
                        df_customers.customer_id,
                        df_customers.customer_name,
                        df_customers.date_of_birth,
                        df_customers.telephone,
                        df_customers.email,
                        df_addresses.address_line_1,
                        df_addresses.city,
                        df_addresses.state,
                        df_addresses.postcode
                    )\
                    .agg(
                        F.countDistinct(df_orders.order_id).alias('total_orders'),
                        F.sum(df_orders.item_quantity).alias('total_items_ordered'),
                        F.sum(df_orders.item_price * df_orders.item_quantity).alias('total_order_amount'))
    return df